In [1]:
import pandas as pd
import numpy as np
import sklearn
from scipy import stats
import lightgbm as lgb
from sklearn.model_selection import ParameterGrid

In [2]:
df_train = pd.read_csv("./Downloads/data/train.csv")
df_val = pd.read_csv("./Downloads/data/validation.csv")
df_test = pd.read_csv("./Downloads/data/test.csv")

In [3]:
X_train = df_train.drop(columns=['Unnamed: 0', 'Date', 'Dep_Var', 'Identifier'])
y_train = df_train['Dep_Var']
X_val = df_val.drop(columns=['Unnamed: 0', 'Date', 'Dep_Var', 'Identifier'])
y_val = df_val['Dep_Var']
X_test = df_test.drop(columns=['Unnamed: 0', 'Date', 'Dep_Var', 'Identifier'])

In [5]:
def val_loss(df,pred,true):
    df['rank'] = 1
    df.sort_values(by=['Date',pred],inplace=True)
    df.loc[df['Date'] == '2014-10-23','pred_sum'] = df.loc[df['Date'] == '2014-10-23','rank'].cumsum()
    df.loc[df['Date'] == '2015-01-21','pred_sum'] = df.loc[df['Date'] == '2015-01-21','rank'].cumsum()
    df.loc[df['Date'] == '2015-04-22','pred_sum'] = df.loc[df['Date'] == '2015-04-22','rank'].cumsum()
    df.loc[df['Date'] == '2015-07-23','pred_sum'] = df.loc[df['Date'] == '2015-07-23','rank'].cumsum()
    df.loc[df['Date'] == '2014-10-23','pred_grp'] = df.loc[df['Date'] == '2014-10-23','pred_sum']/153
    df.loc[df['Date'] == '2015-01-21','pred_grp'] = df.loc[df['Date'] == '2015-01-21','pred_sum']/155
    df.loc[df['Date'] == '2015-04-22','pred_grp'] = df.loc[df['Date'] == '2015-04-22','pred_sum']/157
    df.loc[df['Date'] == '2015-07-23','pred_grp'] = df.loc[df['Date'] == '2015-07-23','pred_sum']/158
    df['pred_grp'] = df['pred_grp'].astype(int)
    return stats.spearmanr(df['pred_grp'],df[true])

#setting parameters for lightgbm
parameter_list = {'num_leaves':[int(x) for x in np.linspace(start=20,stop=500,num=5)],
         'max_depth':[int(x) for x in np.linspace(5,10,5)],
         'n_estimators':[int(x) for x in np.linspace(100,500,5)]}

lgbm_regressor = lgb.LGBMModel(metric='regression_l2',objective='regression')

best_score = 0

for g in ParameterGrid(parameter_list):
    lgbm_regressor.set_params(**g)
    lgbm_regressor.fit(X_train,y_train)
    df_val['y_pred_val'] = lgbm_regressor.predict(X_val)
    spearmanr = val_loss(df_val, 'y_pred_val','Dep_Var')
    # save if best
    if spearmanr.correlation > best_score:
        best_score = spearmanr.correlation
        best_grid = g

In [6]:
best_score

0.02826214645233078